Aluno: Igor Correa Trifilio Campos
Matricula: 202365092AB

In [ ]:
import numpy

In [ ]:

def fekine2d(nnel: int, dhdx: np.ndarray, dhdy: np.ndarray) -> np.ndarray:
    """
    Determina a matriz cinemática que relaciona deformações e deslocamentos
    para sólidos bidimensionais.
    
    Parâmetros:
    -----------
    nnel : int
        Número de nós por elemento
    dhdx : np.ndarray
        Derivadas das funções de forma em relação a x
    dhdy :  np.ndarray
        Derivadas das funções de forma em relação a y
    
    Retorna:
    --------
    kinmtx2 :  np.ndarray
        Matriz cinemática [B] de dimensão (3, 2*nnel)
    """
    
    # Pré-alocação da matriz (3 linhas, 2*nnel colunas)
    kinmtx2 = np.zeros((3, 2 * nnel))
    
    for i in range(nnel):
        # Índices das colunas (ajustados para indexação base-0)
        i1 = 2 * i      # Em MATLAB seria (i-1)*2+1, aqui é 2*i
        i2 = i1 + 1     # Coluna seguinte
        
        # Preenchimento da matriz B
        # Linha 1: ∂u/∂x (deformação εx)
        kinmtx2[0, i1] = dhdx[i]
        
        # Linha 2: ∂v/∂y (deformação εy)
        kinmtx2[1, i2] = dhdy[i]
        
        # Linha 3: ∂u/∂y + ∂v/∂x (deformação γxy)
        kinmtx2[2, i1] = dhdy[i]
        kinmtx2[2, i2] = dhdx[i]
    
    return kinmtx2

In [ ]:
def feeldof(nd: np.ndarray, nnel: int, ndof: int) -> np.ndarray:
    """
    Calcula os graus de liberdade do sistema associados a cada elemento. 
    
    Parâmetros:
    -----------
    nd : np.ndarray
        Vetor com os números dos nós do elemento (conectividade)
    nnel : int
        Número de nós por elemento
    ndof :  int
        Número de graus de liberdade por nó
    
    Retorna: 
    --------
    index : np.ndarray
        Vetor de DOFs globais associados ao elemento
    
    Exemplo:
    --------
    Para um elemento triangular (nnel=3) com nós [1, 3, 2] e ndof=2:
    >>> nd = np.array([1, 3, 2])
    >>> index = feeldof(nd, 3, 2)
    >>> print(index)  # [1, 2, 5, 6, 3, 4] (em notação base-1)
    """
    
    # Número total de DOFs do elemento
    edof = nnel * ndof
    
    # Pré-alocação do vetor de índices
    index = np.zeros(edof, dtype=int)
    
    k = 0  # Contador para posição no vetor index
    
    for i in range(nnel):
        # Calcula o DOF inicial para o nó nd[i]
        # nd[i] - 1 porque a numeração de nós começa em 1
        start = (nd[i] - 1) * ndof
        
        for j in range(ndof):
            # Em Python, index[k] já está em base-0
            # start + j + 1 para manter compatibilidade com numeração base-1 dos DOFs
            index[k] = start + j + 1
            k += 1
    
    return index


def feeldof_pythonic(nd: np.ndarray, nnel: int, ndof:  int) -> np.ndarray:
    """
    Versão alternativa mais Pythonica usando list comprehension.
    Retorna índices em base-0 (convenção Python).
    
    Parâmetros: 
    -----------
    nd : np. ndarray
        Vetor com os números dos nós do elemento (base-1)
    nnel : int
        Número de nós por elemento
    ndof : int
        Número de graus de liberdade por nó
    
    Retorna:
    --------
    index : np.ndarray
        Vetor de DOFs globais em base-0 (convenção Python)
    """
    
    # Usando list comprehension para construção mais elegante
    index = np.array([
        (nd[i] - 1) * ndof + j
        for i in range(nnel)
        for j in range(ndof)
    ], dtype=int)
    
    return index

In [ ]:

def feasmbl1(kk: np.ndarray, k: np.ndarray, index: np.ndarray) -> np.ndarray:
    """
    Montagem (assembly) das matrizes de rigidez elementares na matriz global.
    
    Parâmetros:
    -----------
    kk : np.ndarray
        Matriz de rigidez global do sistema (será modificada)
    k : np.ndarray
        Matriz de rigidez do elemento
    index : np.ndarray
        Vetor de DOFs globais associados ao elemento
    
    Retorna:
    --------
    kk : np.ndarray
        Matriz de rigidez global atualizada
    
    Notas:
    ------
    Esta função assume que 'index' contém índices em base-1 (convenção MATLAB).
    Se usar índices base-0, remova o '-1' nas linhas de ii e jj.
    """
    
    edof = len(index)
    
    for i in range(edof):
        ii = index[i] - 1  # Conversão base-1 → base-0
        for j in range(edof):
            jj = index[j] - 1  # Conversão base-1 → base-0
            kk[ii, jj] = kk[ii, jj] + k[i, j]
    
    return kk


def feasmbl1_vectorized(kk: np.ndarray, k: np.ndarray, index: np.ndarray) -> np.ndarray:
    """
    Versão vetorizada (mais eficiente) da montagem da matriz global.
    
    Parâmetros:
    -----------
    kk : np.ndarray
        Matriz de rigidez global do sistema
    k : np. ndarray
        Matriz de rigidez do elemento
    index :  np.ndarray
        Vetor de DOFs globais (base-0)
    
    Retorna:
    --------
    kk : np.ndarray
        Matriz de rigidez global atualizada
    """
    
    # Cria grade de índices para atribuição vetorizada
    # np.ix_ cria arrays de índices para indexação avançada
    idx = np.ix_(index, index)
    
    # Soma vetorizada - muito mais rápida para grandes sistemas
    kk[idx] += k
    
    return kk


In [ ]:
def feaplyc2(kk: np.ndarray, ff: np.ndarray, 
             bcdof: np. ndarray, bcval: np. ndarray) -> tuple:
    """
    Aplica condições de contorno (deslocamentos prescritos) ao sistema [K]{u}={F}.
    
    Utiliza o método de eliminação:  zera a linha correspondente ao DOF restringido,
    coloca 1 na diagonal e o valor prescrito no vetor de forças.
    
    Parâmetros:
    -----------
    kk : np.ndarray
        Matriz de rigidez do sistema (será modificada)
    ff : np.ndarray
        Vetor de forças do sistema (será modificado)
    bcdof : np.ndarray
        Vetor contendo os DOFs com deslocamento prescrito (base-1)
    bcval : np.ndarray
        Vetor contendo os valores prescritos de deslocamento
    
    Retorna: 
    --------
    tuple :  (kk, ff)
        Matriz e vetor modificados com condições de contorno aplicadas
    
    Exemplo:
    --------
    Para prescrever u=0 nos DOFs 1 e 2, e u=0. 5 no DOF 5:
    >>> bcdof = np.array([1, 2, 5])
    >>> bcval = np.array([0.0, 0.0, 0.5])
    >>> kk, ff = feaplyc2(kk, ff, bcdof, bcval)
    """
    
    n = len(bcdof)        # Número de condições de contorno
    sdof = kk.shape[0]    # Número total de DOFs (dimensão da matriz)
    
    for i in range(n):
        c = bcdof[i] - 1  # DOF restringido (convertido para base-0)
        
        # Zera toda a linha c
        kk[c, : ] = 0
        
        # Coloca 1 na diagonal
        kk[c, c] = 1
        
        # Define o valor prescrito no vetor de forças
        ff[c] = bcval[i]
    
    return kk, ff


def feaplyc2_symmetric(kk: np.ndarray, ff: np.ndarray,
                       bcdof: np.ndarray, bcval: np.ndarray) -> tuple:
    """
    Aplica condições de contorno preservando a simetria da matriz.
    
    Este método é preferível quando se usa solvers que exploram simetria.
    Além de zerar a linha, também zera a coluna e ajusta o vetor de forças. 
    
    Parâmetros:
    -----------
    kk : np.ndarray
        Matriz de rigidez simétrica do sistema
    ff : np.ndarray
        Vetor de forças do sistema
    bcdof : np.ndarray
        Vetor de DOFs restringidos (base-1)
    bcval : np.ndarray
        Vetor de valores prescritos
    
    Retorna: 
    --------
    tuple : (kk, ff)
        Sistema modificado mantendo simetria
    """
    
    n = len(bcdof)
    sdof = kk.shape[0]
    
    for i in range(n):
        c = bcdof[i] - 1  # Conversão para base-0
        prescribed_value = bcval[i]
        
        # Ajusta o vetor de forças para manter simetria
        # ff = ff - kk[: , c] * prescribed_value
        for j in range(sdof):
            if j != c:
                ff[j] = ff[j] - kk[j, c] * prescribed_value
        
        # Zera linha e coluna
        kk[c, :] = 0
        kk[:, c] = 0
        
        # Coloca 1 na diagonal
        kk[c, c] = 1
        
        # Define valor prescrito
        ff[c] = prescribed_value
    
    return kk, ff


def feaplyc2_penalty(kk: np.ndarray, ff: np.ndarray,
                     bcdof: np.ndarray, bcval: np.ndarray,
                     penalty: float = 1e20) -> tuple:
    """
    Aplica condições de contorno pelo método da penalidade.
    
    Adiciona um valor muito grande (penalidade) à diagonal e ao vetor de forças,
    forçando o deslocamento a assumir o valor prescrito.
    
    Parâmetros:
    -----------
    kk : np. ndarray
        Matriz de rigidez do sistema
    ff :  np.ndarray
        Vetor de forças do sistema
    bcdof : np.ndarray
        Vetor de DOFs restringidos (base-1)
    bcval : np.ndarray
        Vetor de valores prescritos
    penalty : float
        Fator de penalidade (default:  1e20)
    
    Retorna:
    --------
    tuple : (kk, ff)
        penalidade
    
    Vantagem:
    ---------
    Mantém a estrutura da matriz original e é mais fácil de implementar
    em códigos com matrizes esparsas.
    """
    
    n = len(bcdof)
    
    for i in range(n):
        c = bcdof[i] - 1  # Conversão para base-0
        
        # Adiciona penalidade à diagonal
        kk[c, c] = kk[c, c] + penalty
        
        # Ajusta vetor de forças
        ff[c] = penalty * bcval[i]
    
    return kk, ff

In [ ]:

def fematiso(iopt:  int, elastic: float, poisson: float) -> np.ndarray:
    """
    Determina a equação constitutiva para material isotrópico.
    
    Parâmetros: 
    -----------
    iopt : int
        Tipo de análise: 
        1 - Estado plano de tensões
        2 - Estado plano de deformações
        3 - Análise axissimétrica
        4 - Análise tridimensional
    elastic : float
        Módulo de elasticidade (E)
    poisson : float
        Coeficiente de Poisson (ν)
    
    Retorna:
    --------
    matmtrx : np.ndarray
        Matriz constitutiva [D]
    """
    
    if iopt == 1:  # Estado plano de tensões
        coef = elastic / (1 - poisson**2)
        matmtrx = coef * np.array([
            [1,       poisson, 0                  ],
            [poisson, 1,       0                  ],
            [0,       0,       (1 - poisson) / 2  ]
        ])
    
    elif iopt == 2:  # Estado plano de deformações
        coef = elastic / ((1 + poisson) * (1 - 2*poisson))
        matmtrx = coef * np.array([
            [(1 - poisson), poisson,       0                    ],
            [poisson,       (1 - poisson), 0                    ],
            [0,             0,             (1 - 2*poisson) / 2  ]
        ])
    
    elif iopt == 3:  # Axissimétrico
        coef = elastic / ((1 + poisson) * (1 - 2*poisson))
        matmtrx = coef * np.array([
            [(1 - poisson), poisson,       poisson,       0                    ],
            [poisson,       (1 - poisson), poisson,       0                    ],
            [poisson,       poisson,       (1 - poisson), 0                    ],
            [0,             0,             0,             (1 - 2*poisson) / 2  ]
        ])
    
    else:  # Tridimensional (iopt == 4)
        coef = elastic / ((1 + poisson) * (1 - 2*poisson))
        matmtrx = coef * np.array([
            [(1-poisson), poisson,     poisson,     0,                  0,                  0                  ],
            [poisson,     (1-poisson), poisson,     0,                  0,                  0                  ],
            [poisson,     poisson,     (1-poisson), 0,                  0,                  0                  ],
            [0,           0,           0,           (1-2*poisson)/2,    0,                  0                  ],
            [0,           0,           0,           0,                  (1-2*poisson)/2,    0                  ],
            [0,           0,           0,           0,                  0,                  (1-2*poisson)/2    ]
        ])
    
    return matmtrx